In [5]:
import pandas as pd
import win32com.client as win32
import datetime
import os.path

In [24]:
# Importing CSV file from Arrow
arrow = pd.read_csv('I:/NI-24/arrow.csv')
arrowe = arrow.loc[arrow['Security Type'] != 'D']
arrowd = arrow.loc[arrow['Security Type'] == 'D']
arrowe = arrowe.loc[~arrowe['Memo 1'].str.startswith("REF#")]
arrowe = arrowe.loc[~arrowe['Memo 1'].str.startswith("REF #")]
arrowf = arrowe.append(arrowd)
arrowf = arrowf.loc[~arrowf['Memo 1'].str.startswith("NO CLIENT")]
arrowf = arrowf.loc[~arrowf['Memo 1'].str.startswith("YIELD")]
arrowf = arrowf.loc[~arrow['Memo 1'].str.startswith("ETF TRADE")]
arrowf = arrowf.loc[~arrowf['Memo 1'].str.startswith(" ")]
arrowf = arrowf.loc[arrowf['Security'] != "CA99997Z1099"]
arrowf = arrowf.loc[arrowf['Trade'] != datetime.datetime.today().strftime('%d-%b-%Y')]
arrowf['Client Account']=arrowf['Client Account'].astype(str)
arrowf

# Importing Blackbook file
blackbook = pd.read_csv('I:/belz/data/Blackbook_v1.1/blackbook_' + datetime.datetime.today().strftime('%Y%m%d') + '.csv')
blackbook['BOI_CODE']=blackbook['BOI_CODE'].astype(str)
blackbook
# Merging Arrow and Blackbook DFs
merge = pd.merge(left=arrowf, right=blackbook[['COUNTERPARTY_CODE', 'BOI_CODE']], how='left', left_on='Client Account', right_on='BOI_CODE')
merge = merge.sort_values(by=['Status','COUNTERPARTY_CODE', 'Security'])
merge = merge.drop_duplicates('Trade Id')
code = merge['COUNTERPARTY_CODE']
merge.drop(labels=['COUNTERPARTY_CODE'], axis=1,inplace = True)
merge.insert(0, 'COUNTERPARTY_CODE', code)
merge = merge.drop(columns='BOI_CODE')
merge["Quantity"]=merge['Quantity'].map('{:,}'.format)
merge["Gross"]=merge['Gross'].map('{:,.2f}'.format)
merge["Net"]=merge['Net'].map('{:,.2f}'.format)
merge.to_csv('I:/NI-24/NI24.csv', encoding='utf-8', index=False)
i = 1
while os.path.isfile('I:/NI-24/ARCHIVE/NI24 ' + datetime.datetime.today().strftime('%Y%m%d') + ' ' + str(i) + '.csv'):
    i = i+1
merge.to_csv('I:/NI-24/ARCHIVE/NI24 ' + datetime.datetime.today().strftime('%Y%m%d') + ' ' + str(i) + '.csv', encoding='utf-8', index=False)
merge = merge.drop(columns='Client Name')
merge
# Creating CSV files with unmatched trades for each client
filtered = merge.loc[merge['Status'] == 'U']
clients = filtered.COUNTERPARTY_CODE.unique()
html = []
for client in clients:
    cpfiltered = filtered.loc[filtered['COUNTERPARTY_CODE'] == client]
    #cpfiltered.to_csv('I:/NI-24/NI24_' + client + '.csv', encoding='utf-8', index=False)
    html.append(cpfiltered[['Trade Id','Match Status','Status','Client Account','Security','Security Name','Role','Quantity','Price','Commission','Gross','Net','Trade','Value','Currency','CUID','Other Account']].to_html())
# Merging Unmatched clients list with NI-24 Email list file
clientsdf = pd.DataFrame({'clients': clients, 'html': html})
emails = pd.read_csv('I:/NI-24/NI-24 emails.csv')
emaillist = pd.merge(left=clientsdf, right=emails, how='left', left_on='clients', right_on='Client')
emaillist
# Sending emails with files
outlook = win32.Dispatch('outlook.application')
for index, row in emaillist.iterrows():
    mail = outlook.CreateItem(0)
    mail.SentOnBehalfOfName = 'Account, NB IE Toronto Support'
    mail.Display()
    signature = mail.HTMLBody
    if pd.notnull(row['Email']): mail.To = row['Email']
    if pd.notnull(row['Cc']): 
        mail.Cc = row['Cc']+'; NBIE.TorontoSupport@bmonb.com'
    else:
        mail.Cc = 'NBIE.TorontoSupport@bmonb.com'
    if row['Email'] == 'David.Morin@bmo.com':
        subject = row['Client'] + ' NI-24 101 Unmatched' 
    else:
        subject = 'NI-24 101 Unmatched'
    mail.Subject = subject
    table = row['html']
    mail.HTMLBody = '<h2><font size="4" style="font-weight:normal">Hi there,<br><br> &nbsp&nbsp&nbsp Please see below and have your custodian match us on CDS as soon as possible.<br>' + table + '<br><br>Thanks,</font></h2><h3>' + signature + '</h2>' #this field is optional
#    attachment  = 'I:\\NI-24\\NI24_' + row[0] + '.csv'
#    mail.Attachments.Add(attachment)
    mail.Display()
outlook = None
excel = win32.Dispatch('excel.application')
excel.WorkBooks.Open('I:/NI-24/NI24.csv')
excel = None